In [1]:
import os
import io
import platform
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.datasets as datasets
import random
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image
from PIL import ImageOps

In [2]:
desired_width = 224
transform=transforms.ToTensor()
def process_img(image):
    desired_length = int(desired_width * (image.size[0]/image.size[1]))
    new=image.resize((desired_width, desired_width))
    back = Image.new("L", (224, 224))  
    back.paste(new)
    img=transform(back)
    new.close()
    back.close()
    return img

In [6]:
FILE_PATH="../chest-xray-pneumonia/chest_xray/chest_xray/"
def load_data(dataset):
    path=FILE_PATH+dataset+"/"
    data=[]
    for file in os.listdir(path+"PNEUMONIA/"):
        if not(file.startswith('.')):
            img=Image.open(path+"PNEUMONIA/"+file)
            temp=[]
            temp.append(process_img(img.convert("L")))
            temp.append(1)
            data.append(temp)
            img.close()
    for file in os.listdir(path+"NORMAL/"):
        if not(file.startswith('.')):
            img=Image.open(path+"NORMAL/"+file)
            temp=[]
            temp.append(process_img(img.convert("L")))
            temp.append(0)
            data.append(temp)
            img.close()
    return data

In [7]:
train_set=load_data("train")
test_set=load_data("test")
val_set=load_data("val")

In [8]:
train_set[0][0]

tensor([[[0.0706, 0.0824, 0.1059,  ..., 0.0314, 0.0078, 0.0039],
         [0.0745, 0.0941, 0.1059,  ..., 0.0353, 0.0196, 0.0039],
         [0.0863, 0.0941, 0.0980,  ..., 0.0392, 0.0235, 0.0039],
         ...,
         [0.0392, 0.0392, 0.0353,  ..., 0.0588, 0.0588, 0.0588],
         [0.0392, 0.0392, 0.0392,  ..., 0.0588, 0.0588, 0.0588],
         [0.0392, 0.0392, 0.0392,  ..., 0.0588, 0.0588, 0.0588]]])

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cl1=nn.Conv2d(1,12,kernel_size=3)
        self.mp1=nn.MaxPool2d(2)
        self.cl2=nn.Conv2d(12,3,kernel_size=3)
        self.mp2=nn.MaxPool2d(2)
        self.fcl=nn.Linear(8748,2)
        
    def forward(self,x):
        # x dim: b x 800 x 600 x 3 -> b x 1 x 100 x 100
        out=F.relu(self.cl1(x)) # b x 18 x whatever x whatever
        out=self.mp1(out)
        out=F.relu(self.cl2(out))
        out=self.mp2(out)
        
        # at this point, we should have a tensor that's b x inchannels x 1 x 1
        out=out.view(out.size(0),-1)
#         print(out.shape)
        out=F.sigmoid(self.fcl(out))
        return out

In [10]:
model=CNN()
model.eval()
optimizer=optim.Adam(model.parameters(),lr=0.001,weight_decay=0.00001)
criterion=nn.CrossEntropyLoss()

In [11]:
def train(epoch,model,train_loader,optimizer):
    model.train()
    
    total_loss=0
    correct=0
    
    for i, (image,label) in enumerate(train_loader):
        
        #print(i)
        optimizer.zero_grad()
        
#         print(label.shape)
       
        
        prediction=model(image)
        
        loss=criterion(prediction,label)
        
        loss.backward()
        
        optimizer.step()
        
        total_loss+=loss
        
        pred_classes=prediction.data.max(1,keepdim=True)[1]
        
        correct+=pred_classes.eq(label.data.view_as(pred_classes)).sum().double()
        
    mean_loss=total_loss/len(train_loader.dataset)
    acc=correct/len(train_loader.dataset)
    
    print('Train Epoch: {}   Avg_Loss: {:.5f}   Acc: {}/{} ({:.3f}%)'.format(
        epoch, mean_loss, correct, len(train_loader.dataset),
        100. * acc))

    return mean_loss, acc

In [12]:
def eval(model,test_loader):
    model.eval()
    
    total_loss=0
    correct=0
    
    for i, (image,label) in enumerate(test_loader):
        
        optimizer.zero_grad()
        
        prediction=model(image)
        
        loss=criterion(prediction,label)
        
        loss.backward()
        
        optimizer.step()
        
        total_loss+=loss
        
        pred_classes=prediction.data.max(1,keepdim=True)[1]
        
        correct+=pred_classes.eq(label.data.view_as(pred_classes)).sum().double()
        
    mean_loss=total_loss/len(test_loader.dataset)
    
    acc=correct/len(test_loader.dataset)
    
    print('Eval:   Avg_Loss: {:.5f}   Acc: {}/{} ({:.3f}%)'.format(
         mean_loss, correct, len(test_loader.dataset),
        100. * acc))

    return mean_loss, acc

In [13]:
def save_model(epoch, model, path='./'):
    
    # file name and path 
    filename = path + 'chestXRay_{}.pt'.format(epoch)
    
    # load the model parameters 
    torch.save(model.state_dict(), filename)
    
    
    return model

def load_model(epoch, model, path='./'):
    
    # file name and path 
    filename = path + 'chestXRay_{}.pt'.format(epoch)
    
    # load the model parameters 
    model.load_state_dict(torch.load(filename))
    
    
    return model

In [14]:
BATCH_SIZE=32
train_loader=data.DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
valid_loader=data.DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=True)
test_loader=data.DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [15]:
train_loader

In [ ]:
num_epoch=10
checkpoint_freq=5
path="./"

train_losses = []
test_losses = []

train_accuracies = []
test_accuracies = []

# traininng 
for epoch in range(1, num_epoch + 1):
    
    # train() function (see above)
    train_loss, train_acc = train(epoch, model, train_loader, optimizer)
    
    # eval() functionn (see above)
    test_loss, test_acc = eval(model, test_loader)    
    
    # append lists for plotting and printing 
    train_losses.append(train_loss)    
    test_losses.append(test_loss)
    
    train_accuracies.append(train_acc)    
    test_accuracies.append(test_acc)
    
    # Checkpoint
    if epoch % checkpoint_freq ==0:
        save_model(epoch, model, path)

# Last checkpoint
save_model(num_epoch, model, path)
    
print("\n\n\nOptimization ended.\n") 

C:\Users\Kaylee\Anaconda3\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1   Avg_Loss: 0.01652   Acc: 4002.0/5216 (76.725%)
Eval:   Avg_Loss: 0.01494   Acc: 534.0/624 (85.577%)
Train Epoch: 2   Avg_Loss: 0.01147   Acc: 4948.0/5216 (94.862%)
Eval:   Avg_Loss: 0.01417   Acc: 540.0/624 (86.538%)
Train Epoch: 3   Avg_Loss: 0.01103   Acc: 5017.0/5216 (96.185%)
Eval:   Avg_Loss: 0.01362   Acc: 555.0/624 (88.942%)
Train Epoch: 4   Avg_Loss: 0.01095   Acc: 5019.0/5216 (96.223%)
Eval:   Avg_Loss: 0.01493   Acc: 518.0/624 (83.013%)
Train Epoch: 5   Avg_Loss: 0.01087   Acc: 5036.0/5216 (96.549%)
Eval:   Avg_Loss: 0.01354   Acc: 552.0/624 (88.462%)


In [ ]:
cnn=CNN()
cnn=load_model(10,cnn)

In [ ]:
cnn

In [ ]:
import matplotlib.pyplot as plt
losses=[]
total_loss=0
criterion=nn.CrossEntropyLoss()
for i,(image,label) in enumerate (val_set):
#     print(image.shape)
    predictio=cnn(image.unsqueeze(0))
    loss=criterion(prediction,label)
    losses.append(loss)
    total_loss+=loss
avg_loss=total_loss/len(val_set.dataset)
print(avg_loss)

In [ ]:
plt.plot(losses,color='darkcyan')
plt.show()

In [ ]:
 @app.route('/manage_game', methods=['POST'])
    import cgi
    form = cgi.FieldStorage()
    upload_img =  form.getvalue('pic')
     
    desired_width = 224
    transform=transforms.ToTensor()

    desired_length = int(desired_width * (upload_img.size[0]/upload_img.size[1]))
    new=upload_img.resize((desired_width, desired_width))
    back = Image.new("L", (224, 224))  
    back.paste(new)
    img=transform(back)
    new.close()
    back.close()
    
    prediction = model(img)
    
    print(prediction)
    
#     print("""
#     <!DOCTYPE html>
#     <html>
#     <head>
#         <title>Page Title</title>
#         <link rel="stylesheet" href="demo.css">
#     </head>

#     <body>

#     <div class="head-part">
#         <div class="head-word">
#             <div class="header">
#                 <p>Pneumonia Detector <span>Demo</span></p>
#             </div>
#             <divss class="description">
#                 <p>An easy-used, reliable tool to help identify whether the lung
#                     has pneumonia, with simply one uploaded chest x-ray.</p>
#             </divss>
#         </div>


#     </div>

#       """)
          
    